In [9]:
import pandas as pd

In [10]:
# Create a set to store the column names
column_names = set()

# For each sheet in the Excel file, print the column names
for sheet in pd.read_excel('./data/input/New Customers.xlsx', sheet_name=None):
    # Show the sheet name and the column names for this sheet
    print('Sheet Name:', sheet)
    sheet_column_names = list(pd.read_excel('./data/input/New Customers.xlsx', sheet_name=sheet).columns)
    print('Column Names:', sheet_column_names)
    print('='*50)
    # Add the column names to the set
    column_names.update(sheet_column_names)

# Print the set of column names
print('Column Names:', column_names)

Sheet Name: January
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: February
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: March
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: April
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: May
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: June
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: July
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: August
Column Names: ['ID', 'Joining Day', 'Demographiic', 'Value']
Sheet Name: September
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: October
Column Names: ['ID', 'Joining Day', 'Demagraphic', 'Value']
Sheet Name: November
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Sheet Name: December
Column Names: ['ID', 'Joining Day', 'Demographic', 'Value']
Column Names: {'Demographic', 'ID', 'Demagraphic'

# Preprocessing

In [11]:
rename_fields = {
    'Demographiic' : 'Demographic', 
    'Demagraphic' : 'Demographic'
}

In [23]:
# Open the Excel files in one concatenated dataframe
with pd.ExcelFile('./data/input/New Customers.xlsx') as f:
    df = pd.concat(
        [ ( pd.read_excel(f, sheet_name=sheet)
            .assign(Month=sheet)
            .rename(columns=rename_fields)
        )
        for sheet in f.sheet_names]
    )

df.head()

,ID,Joining Day,Demographic,Value,Month
0,490910,3,Ethnicity,White,January
1,490910,3,Date of Birth,5/23/1981,January
2,490910,3,Account Type,Basic,January
3,369221,18,Ethnicity,Black,January
4,369221,18,Date of Birth,3/4/2019,January


In [40]:
( df
    # Make a Joining Date field based on the fields Joining Day and Month. The year is 2023
    .assign(Joining_Date = lambda x: pd.to_datetime('2023-' + x['Month'].astype(str) + '-' + x['Joining Day'].astype(str)))
    # Reshape the Demographic field into columns - the column names should be the values in the Demographic field
    .pivot_table(index=['ID', 'Joining_Date'], columns='Demographic', values='Value', aggfunc='first')
    .reset_index()
    # Remove any duplicate rows based on ID and Joining Date, keeping the first record
    .sort_values(['Joining_Date', 'ID'])
    .drop_duplicates(subset=['ID'], keep='first')
    # Remove the index name and reset it
    .rename_axis(None, axis=1)
    .reset_index(drop=True)
    # Change the data types of the fields to match the data dictionary
    .astype({
        'ID': 'int64',
        'Joining_Date': 'datetime64[ns]',
        'Account Type': 'category',
        'Date of Birth': 'datetime64[ns]',
        'Ethnicity': 'category'
    })
    # Format the date fields in the DD/MM/YYYY format
    .assign(
        Joining_Date = lambda x: x['Joining_Date'].dt.strftime('%d/%m/%Y'),
        Date_of_Birth = lambda x: x['Date of Birth'].dt.strftime('%d/%m/%Y')
    )
    .drop(columns=['Date of Birth'])
    .rename(columns={
        'Date_of_Birth': 'Date of Birth',
        'Joining_Date': 'Joining Date'
    })
    .to_csv(path_or_buf='./data/output/output_2023_04_.csv', index=False, quoting=1, quotechar='"', sep=';')
)